# Some classification benchmarks

I have a feeling that this isn't going to do so great, because chirping motif can pretty much be in any position. This would work better if I could extract the similarity scores from a representative sample from each class. I do need the methology in the future, so this isn't entirely in vain.

In [1]:
from pathlib import Path
import numpy as np
import IPython.display as ipd
import librosa


def cens_per_sec(sample_rate, target):
    """Ensure this value is a multiple of 2**6"""
    return (sample_rate // (target * (2 ** 6))) * (2 ** 6)

def chroma_cens(data, sample_rate=22050, cens_sr=10):
    return librosa.feature.chroma_cens(
        data, sample_rate, hop_length=cens_per_sec(sample_rate, cens_sr)
    )

def normalize(row):
    if row.shape[0] == 600:
        return row
    if row.shape[0] > 600:
        return row[:600]
    if row.shape[0] < 600:
        return np.append(row, np.zeros(600-row.shape[0]))

    
root = Path("../data")
motif = root / "motif/train_short_audio"

data = []
species = []
audio = []
for path in motif.glob("**/*.npy"):
    data.append(np.load(path))
    species.append(path.parent.parent.name)
    audio.append(path.as_posix().replace(".npy", ".ogg"))

# TODO: let's write out the cens data to file too, it's slow here
chroma = [chroma_cens(row).reshape(-1) for row in data]
X = np.stack([normalize(r) for r in chroma])

In [2]:
X.shape

(592, 600)

In [6]:
from matplotlib import pyplot as plt
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(species)
y = le.transform(species)

In [24]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y
)

clf = LogisticRegression(max_iter=500).fit(X_train, y_train)
clf.score(X_test, y_test)

0.5546218487394958

Yeah, that's pretty awful.